Lib imports

In [2]:
from gx_utils import *
import logging
import numpy as np
import random
from random import seed, choice
from typing import Callable

logging.basicConfig(format="%(message)s", level=logging.INFO)

Problem definition - lab delivery

In [3]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

Professor's solution - lab delivery

In [ ]:
def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    print(all_lists, " all lists")
    while goal != covered:
        x = all_lists.pop(0)
        print(x, " x")
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")
greedy(5)

In [ ]:
GOAL = (np.array(range(5)))
logging.info(f"Goal:\n{GOAL}")
print(type(GOAL))

def goal_test(state):
    return state == GOAL

In [ ]:
INITIAL_STATE = problem(5,42)
print(len(INITIAL_STATE)) #quante parentesi
print((INITIAL_STATE))

Greedy solution

In [ ]:
def set_covering_problem_greedy(universe,subsets,costs):
    cost=0
    visited=0
    #create a unique set - source: documentation
    elements=set(e for s in subsets for e in s) 
    if elements!=universe:
        return None
    covered=set()
    solution=[]
    while covered!=elements:
        subset = max(subsets,key=lambda s: len(s-covered)/costs[subsets.index(s)])
        solution.append(subset)
        cost+=costs[subsets.index(subset)]
        visited=visited+1
        covered |= subset
    print("NUMBER OF VISITED NODES: ",visited)
    print("w: ",sum(len(_) for _ in solution))
    return solution,cost 

def start(N):
    """ Start: 
        N: N
        I suppose cost are unit
    """
    sub=problem(N,seed=42)
    #all costs are 1 
    C=np.full(len(sub),1)
    #print("C ",C) #all 1
    universe=set(range(N))
    #create a unique set - source: documentation
    subsets=[set(element) for element in sub]
    costs=C 
    set_cover,totalCosts=set_covering_problem_greedy(universe,subsets,costs)
    print("UNIVERSE: ",universe)
    print("COVERING SETS: ",set_cover)
    print("COST: ",totalCosts)



#N=5
print("5")
start(5)
print("----------------------------------------------------------------")
print("10")
start(10)
print("----------------------------------------------------------------")
print("20")
start(20)
print("----------------------------------------------------------------")
print("50")
start(50)
print("----------------------------------------------------------------")
print("100")
start(100)
print("----------------------------------------------------------------")
print("500")
start(500)
print("----------------------------------------------------------------")
print("1000")
start(1000)



Breadth first solution

In [ ]:
def set_covering_problem_bf(universe,subsets,costs):
    cost=0
    elements=set(e for s in subsets for e in s)
    if elements!=universe:
        print("The subsets don't contain the universe.")
        return None
    covered=set()
    solution=[]
    queue=[]
    visited=[]
    while covered!=elements:
        subset = max(subsets,key=lambda s: len(s-covered))
        #print(subset, " subset")
        queue.append(subset)
        x=queue.pop()
        if x not in visited:
            visited.append(subset)
            solution.append(subset)
            cost+=costs[subsets.index(subset)]
            covered |= subset
    print("NUMBER OF VISITED NODES: ",len(visited))
    print("w: ",sum(len(_) for _ in solution))
    return solution,cost

def start(N):
    """ Start: 
        N: N
        I suppose cost are unit
    """
    sub=problem(N,seed=42)
    #all costs are 1 
    C=np.full(len(sub),1)
    #print("C ",C)
    universe=set(range(N))
    #create a unique set - source: documentation
    subsets=[set(element) for element in sub]
    costs=C 
    set_cover,totalCosts=set_covering_problem_bf(universe,subsets,costs)
    print("UNIVERSE: ",universe)
    print("COVERING SETS: ",set_cover)
    print("COST: ",totalCosts)


#N=5
print("5")
start(5)
print("----------------------------------------------------------------")
print("10")
start(10)
print("----------------------------------------------------------------")
print("20")
start(20)
print("----------------------------------------------------------------")
print("50")
start(50)
print("----------------------------------------------------------------")
print("100")
start(100)
print("----------------------------------------------------------------")
print("500")
start(500)
print("----------------------------------------------------------------")
print("1000")
start(1000)
